**ภาพรวม**
- คลาส CharMovement1 จัดการการเคลื่อนที่ของตัวละคร รวมถึงการเดินและการกระโดด พร้อมกับอนิเมชันที่เกี่ยวข้องและการทำงานของแรงโน้มถ่วง

**ส่วนประกอบหลัก**
1. ตัวแปร:

- พารามิเตอร์การเคลื่อนที่:
  - speed: กำหนดความเร็วในการเคลื่อนที่ของตัวละคร
  - jumpForce: ควบคุมความสูงในการกระโดด
  - gravity: ส่งผลต่อแรงดึงดูดลงสู่พื้น
  - rotationSpeed: กำหนดความเร็วในการหมุนของตัวละครไปในทิศทางที่เคลื่อนที่
- สถานะอนิเมชัน:
  - ตัวแปรบูลีน เช่น isGrounded, isDef, isDancing, และ isWalking ใช้ติดตามการกระทำของตัวละคร
- ตัวแปรภายใน:
  - animator: อ้างอิงถึงคอมโพเนนต์ Animator เพื่อจัดการอนิเมชัน
  - characterController: อ้างอิงถึง CharacterController สำหรับการเคลื่อนที่
เวกเตอร์สำหรับการเคลื่อนที่และการจัดการอินพุต

**วิธีการใน Unity**:

- Awake(): เริ่มต้นคอมโพเนนต์ characterController และ animator
- Start(): ตั้งค่าสเกลเวลาในเกมและตรวจสอบว่าตัวละครอยู่บนพื้นหรือไม่

**วิธีการ Update:**

- รับอินพุตจากผู้เล่นโดยใช้ Input.GetAxis() สำหรับการเคลื่อนที่ในแนวนอนและแนวตั้ง
- อัปเดตพารามิเตอร์อนิเมชันตามอินพุต
- ตรวจสอบว่าตัวละครกำลังเดินอยู่หรือไม่และตั้งค่าตัวแปร isWalking
- จัดการการกระโดด (ปัจจุบันยังไม่เปิดใช้งาน) และใช้แรงโน้มถ่วงในการเคลื่อนที่

**ลอจิกการเคลื่อนที่:**

- หากตัวละครอยู่บนพื้น จะคำนวณทิศทางการเคลื่อนที่และนำความเร็วไปใช้
- ใช้แรงโน้มถ่วงเพื่อให้ตัวละครอยู่บนพื้น

**วิธีช่วย:**

- updateMovement(): ปรับขนาดอินพุตการเคลื่อนที่และจัดการการหมุนและทิศทางตามมุมมองของกล้อง
- rotatTowardMovement(): หมุนตัวละครไปในทิศทางที่เคลื่อนที่โดยใช้การปั้นกลม (spherical interpolation)
- viewRelativeMovement(): คำนวณทิศทางการเคลื่อนที่ที่สัมพันธ์กับทิศทางของกล้อง

**การรวมอนิเมชัน**
- โค้ดนี้รวมกับระบบ Animator ของ Unity เพื่อควบคุมอนิเมชันตามอินพุตจากผู้เล่น ทำให้มีการตอบสนองที่ไดนามิกต่อการกระทำของผู้เล่น

**การปรับปรุงที่เป็นไปได้**
- ลอจิกการกระโดด: เปิดใช้งานและปรับปรุงลอจิกการกระโดดเพื่อให้ตัวละครสามารถกระโดดได้เมื่ออยู่บนพื้น
- การจัดการสถานะ:
  ใช้การนับค่าหมายเลขสำหรับสถานะของตัวละครแทนการใช้ตัวแปรบูลีนหลายตัวเพื่อความชัดเจนและการบำรุงรักษาที่ดีขึ้น
- การจัดการอินพุต: ใช้ระบบอินพุตใหม่ของ Unity สำหรับการจัดการอินพุตที่ซับซ้อนมากขึ้น

**สรุป**
- โค้ดนี้ให้พื้นฐานที่ดีสำหรับการเคลื่อนที่ของตัวละครใน Unity ครอบคลุมกลไกพื้นฐานเช่นการเดินและการกระโดด พร้อมกับการหมุนที่ราบรื่นและการรวมอนิเมชัน สามารถขยายหรือปรับปรุงเพิ่มเติมตามความต้องการของเกม

**Code:**

In [ ]:
using System;
using System.Collections;
using System.Collections.Generic;
using Unity.VisualScripting;
using UnityEngine;

public class CharMovement1 : MonoBehaviour
{
    // พารามิเตอร์การเคลื่อนที่ของตัวละคร
    public float speed = 10.0f;          // ความเร็วในการเคลื่อนที่
    public float jumpForce = 8.0f;       // แรงกระโดด
    public float gravity = 20.0f;         // แรงโน้มถ่วง
    public float rotationSpeed = 100.0f;  // ความเร็วในการหมุน

    // สถานะการอนิเมชัน
    public bool isGrounded = false;       // ตัวละครอยู่บนพื้นหรือไม่
    public bool isDef = false;             // สถานะป้องกัน
    public bool isDancing = false;         // สถานะเต้น
    public bool isWalking = false;         // สถานะเดิน

    private Animator animator;             // อ้างอิงถึง Animator
    private CharacterController characterController; // อ้างอิงถึง CharacterController
    private Vector3 inputVector = Vector3.zero;  // เวกเตอร์อินพุต
    private Vector3 targetDirection = Vector3.zero; // ทิศทางเป้าหมาย
    private Vector3 moveDirection = Vector3.zero; // ทิศทางการเคลื่อนที่
    private Vector3 velocity = Vector3.zero;       // ความเร็ว

    void Awake()
    {
        // เริ่มต้นคอมโพเนนต์
        characterController = GetComponent<CharacterController>();
        animator = GetComponent<Animator>();
    }

    void Start()
    {
        Time.timeScale = 1; // ตั้งค่าสเกลเวลาในเกม
        isGrounded = characterController.isGrounded; // ตรวจสอบว่าตัวละครอยู่บนพื้น
    }

    void Update()
    {
        // รับค่าการเคลื่อนที่จากผู้เล่น
        float z = Input.GetAxis("Horizontal");
        float x = -(Input.GetAxis("Vertical"));

        // อัปเดตพารามิเตอร์อนิเมชัน
        animator.SetFloat("inputX", -(x));
        animator.SetFloat("InputZ", z);

        // ตรวจสอบว่าตัวละครกำลังเดินหรือไม่
        if (x != 0 || z != 0)
        {
            isWalking = true;
            animator.SetBool("isWalking", isWalking);
        }
        else
        {
            isWalking = false;
            animator.SetBool("isWalking", isWalking);
        }

        // ตรวจสอบว่าตัวละครอยู่บนพื้น
        isGrounded = characterController.isGrounded;
        if (isGrounded)
        {
            moveDirection = new Vector3(Input.GetAxis("Horizontal"), 0.0f, Input.GetAxis("Vertical"));
            moveDirection *= speed;

            // เปิดใช้งานการกระโดดหากผู้เล่นกดปุ่มกระโดด (ยังไม่เปิดใช้งาน)
            // if (Input.GetButton("Jump"))
            // {
            //     moveDirection.y = jumpForce;
            // }
        }

        // ใช้แรงโน้มถ่วงสำหรับตัวละคร
        moveDirection.y -= gravity * Time.deltaTime;

        // เคลื่อนที่ตัวละคร
        characterController.Move(moveDirection * Time.deltaTime);

        // อัปเดตการเคลื่อนที่
        inputVector = new Vector3(x, 0, z);
        updateMovement();
    }

    void updateMovement()
    {
        Vector3 motion = inputVector;
        // ปรับขนาดอินพุตการเคลื่อนที่
        motion = ((Mathf.Abs(motion.x) > 1) || (Mathf.Abs(motion.z) > 1)) ? motion.normalized : motion;

        // หมุนตัวละครไปในทิศทางที่เคลื่อนที่
        rotatTowardMovement();
        viewRelativeMovement();
    }

    void rotatTowardMovement()
    {
        // หมุนตัวละครไปในทิศทางที่เคลื่อนที่
        if (inputVector != Vector3.zero)
        {
            Quaternion targetRotation = Quaternion.LookRotation(targetDirection);
            transform.rotation = Quaternion.Slerp(transform.rotation, targetRotation, rotationSpeed * Time.deltaTime);
        }
    }

    void viewRelativeMovement()
    {
        // คำนวณการเคลื่อนที่ตามมุมมองของกล้อง
        Transform cameraTransform = Camera.main.transform;
        Vector3 forward = cameraTransform.TransformDirection(Vector3.forward);
        forward.y = 0.0f;
        forward = forward.normalized;
        Vector3 right = new Vector3(forward.z, 0.0f, -forward.x);
        targetDirection = (Input.GetAxis("Horizontal") * right) + (Input.GetAxis("Vertical") * forward);
    }
}